In [5]:
# run this if game crashes and you want to see documentation with shift/tab
import pygame
import numpy as np
import matplotlib.pyplot as plt
import time
import random
import sys

# RL Tetris Project Outline:
0) Clean up code
    - separate classes from main loop and RL components (easier to run 1 cell at the moment)
    - remove legacy comments
    - make production ready
1) [x] Create figure generator
    - [x] Choose one of 7 blocks (O, I, S, Z, J, L, T)
        - [x] Figure out color for block
        - [x] List out rotations for each block
2) Generate board/figure
    - [x] Adjust general layout for board
        - timer
        - [x] score display
        - [x] lines display
        - level display 
    - Implement buttons (not essential atm)
        - new game button (esc for now)
        - reset button
        - pause button
    - Implement debugger/ replay buff
        - back (last n = 5 blocks, and game states, probably useful for debugging)
        * need to store matrix, queue, swap piece and time? Reset all. Not sure how to deal with other saved states?
    - [x] area for next blocks (show up to k=3)
    - [x] area for swapping blocks
3) Game logic
    - clean up timer for piece movement
        - add timer display for over all time (and to potentially calculate reward function)
    - [x] set up block shadow where piece will land (useful for computer mode)
    - Set up controls for piece movement
        - [x] human mode: button presses
        - computer mode (render or not)
    - [x] make piece generator function
        - [x] go to next piece queue to display
        - [x] previous next piece gets bumped to board (above game)
    - [x] make movement functions
        - [x] left, right, soft drop, hard drop
        - [x] CW rotate, CCW rotate
        - [x] kick checks for rotations (only checked example shown here: https://tetris.fandom.com/wiki/SRShttps://tetris.fandom.com/wiki/SRS, HARD to check (I'm not a skilled Tetris player)!
    - make update screen function
        - [x] move piece
        - check for silver/golden squares
        - check for line clears
            - drop pieces above (already implemented in original code)
            - [x] adjust score (could be updated)
            - [x] adjust lines
            - adjust level
            - play sound
           
4) RL Algorithm
    - review articles on making tetris and how to avoid getting stuck with delayed reward
    - research if possible to help model train by giving examples (showing how to clear lines, probably slow)
    - create option for human input and computer with rendering (allow policy to be chosen, random or model)
        - Make computer play random moves sampling from action space of allowable moves
        - Eventually swap to play best model moves and render
        - Develop algorithm to figure out 'all' (maybe most since getting every landing seems super difficult) landing spots and moves to get there
    - Create OpenAI gym tetris environment
    - Train model
        - Test different action spaces
            - Simple movements
            - complex movements
        - test different algorithms
            - double q
            - PPO
            - add more here...
        - test if adding heuristics as features improves convergence
    - PROBABLY MUCH MORE TO DO IN THIS SECTION
5) Streamlit App bonus
    - allow user to play vs computer


In [22]:
### Code originally taken from:
# https://levelup.gitconnected.com/writing-tetris-in-python-2a16bddb5318
# highly modified
import pygame
import random
import sys


# Define some colors
BLACK = (0, 0, 0)
WHITE = (255, 255, 255)
GRAY = (128, 128, 128)
TRANSPARENCY = 50
size = (800, 800)
        
colors = [
    (255, 255, 255), # EMPTY - WHITE
    (255, 215, 0), # GOLD 
    (194, 189, 176), # SILVER
    (0, 255, 255), # I- Cyan
    (0, 0, 255), # J - Blue
    (255, 127, 0), # L - Orange
    (255, 255, 0), # O - Yellow
    (0, 255, 0), # S - Green 
    (128, 0, 128), # T - Purple
    (255, 0, 0), # Z - Red  
]




class Figure:
    # x,y are relative to game board field
    x = 0
    y = 0

    names = ['I', 'J', 'L', 'O', 'S', 'T', 'Z']
    figures = [
        [[4, 5, 6, 7], [2, 6, 10, 14], [8,9,10,11], [1,5,9,13]], # I
        [[0, 4, 5, 6], [1, 2, 5, 9], [4, 5, 6, 10], [1, 5, 9, 8]], # J
        [[4, 5, 6, 2], [1, 5, 9, 10], [8, 4, 5, 6], [0, 1, 5, 9]], # L
        [[1, 2, 5, 6]], # O
        [[4, 5, 1, 2], [1, 5, 6, 10], [8, 9, 5, 6], [0, 4, 5, 9]], # S
        [[1, 4, 5, 6], [1, 5, 6, 9], [4, 5, 6, 9], [1, 4, 5, 9]], # T
        
        [[0, 1, 5, 6], [2, 6, 5, 9], [4, 5, 9, 10], [1, 5, 4, 8]] # Z   
    ]
    
    # Used for rotational nudges
    # Used for I, J, T, S, Z blocks (some T blocks won't be achieved)
    # O blocks cannot rotate
    kicks_main = {
        (0,1): [(-1,0), (-1,1), (0,-2), (-1,-2)],
        (1,0): [(1,0), (1,-1), (0,2), (1,2)],
        (1,2): [(1,0), (1,-1), (0,2), (1,2)],
        (2,1): [(-1,0), (-1,1), (0,-2), (-1,-2)], 
        (2,3): [(1,0), (1,1), (0,-2), (1,-2)],
        (3,2): [(-1,0), (-1,-1), (0,2), (-1,2)],
        (3,0): [(-1,0), (-1,-1), (0,2), (-1,2)],
        (0,3): [(1,0), (1,1), (0,-2), (1,-2)]
    }

    # Different set of nudges for I blocks
    kicks_I  = {
        (0,1): [(-2,0), (1,0), (-2,-1), (1,2)],
        (1,0): [(2,0), (-1,0), (2,1), (-1,-2)],
        (1,2): [(-1,0), (2, 0), (-1,2), (2,-1)],
        (2,1): [(1,0), (-2,0), (1,-2), (-2, 1)], 
        (2,3): [(2,0), (-1,0), (2,1), (-1,-2)],
        (3,2): [(-2,0), (1,0), (-2,-1), (1,2)],
        (3,0): [(1,0), (-2,0), (1,-2), (-2,1)],
        (0,3): [(-1,0), (2,0), (-1,2), (2,-1)]
    }
        

    def __init__(self, x, y, mode = None):
        self.x = x
        self.y = y
        if not (mode in range(len(self.figures))):
            self.type = random.randint(0, len(self.figures) - 1)
        else:
            self.type = mode
        self.color = self.type + len(colors) - len(self.figures)
        self.name = self.names[self.type]
        self.rotation = 0

    def image(self):
        return self.figures[self.type][self.rotation]

    def rotate(self, direction):
        self.rotation = (self.rotation + direction) % len(self.figures[self.type])

class Tetris:
    level = 1 # used for speed, doesn't update yet
    score = 0
    buffer = 4
    n_queue = 3 # how many next tetrominos to show
    mode = 'normal' 
    field = []
    # height = 0 # defined in the init method
    # width = 0
    x = 200
    y = 100
    
    # tetromino starting coordinates with buffer, 4 right, 2 down
    tet_x = 4
    tet_y = 2
    
    # swap coordinates for left side of board
    swap_x = 50
    swap_y = 200
    
    # Queue coordinates top piece
    queue_x = 500
    queue_y = 250
    zoom = 25 # size of grid squares
    
    def __init__(self, height=20, width=10):
        self.height = height
        self.full_height = self.height + self.buffer
        self.width = width
        self.field = []
        self.score = 0
        self.lines = 0
        self.state = "start"
        self.queue = []
        self.has_swapped = False # to allow user to save a tetromino to swap for later, once per drop
        self.swap_piece = None
        
        # Set waiting queue and current figure
        for i in range(self.n_queue):
            self.queue.append(Figure(x=self.tet_x,  y=self.tet_y))

        self.figure =  Figure(x=self.tet_x,  y=self.tet_y)
        
        
        
        
        
        for i in range(self.full_height): # add rows above game matrix
            new_line = []
            for j in range(width):
                new_line.append(0)
                # TESTING KICKS, this example: https://tetris.fandom.com/wiki/SRS:
                # if (i,j) in [(23,0),(23,1), (23,2), (23,3), (23,4), (23,6), (23,7), (23,8), (23,9),
                #              (22,0),(22,1), (22,2), (22,3), (22,6), (22,7), (22,8), (22,9),
                #              (21,0),(21,1), (21,6), (21,7), (21,8), (21,9),
                #              (20,1), (20,2), (20,3), (20,7), (20,8), (20,9),
                #              (19,6), (19,7), (19,8), (19,9),
                #              (18,6), (18,7), (18,5),
                #              (17,4), (17,5)]:
                #     new_line.append(1)
                # else:
                #     new_line.append(0)
            self.field.append(new_line)

    def new_figure(self, mode=None):
        # if short on blocks, add them to queue
        # pop the first one from the list, add another to end
        while len(self.queue) < self.n_queue:
            self.queue.append(Figure(x=self.tet_x,  y=self.tet_y))
        if mode is None:
            self.figure = self.queue.pop(0)
            self.queue.append(Figure(x=self.tet_x,  y=self.tet_y))
        else:
            self.figure = Figure(x=self.tet_x,  y=self.tet_y, mode=mode)
        

    def intersects(self):
        intersection = False
        for ind in self.figure.image():
            i = ind//4
            j = ind%4
            # check was > 0, but I want negative numbers to eventually check if block has been broken for gold/silver squares
            if i + self.figure.y >= self.full_height or \
                    j + self.figure.x >= self.width or \
                    j + self.figure.x < 0 or \
                    self.field[i + self.figure.y][j + self.figure.x] != 0: 
                intersection = True
        return intersection

    def break_lines(self):
        lines = 0
        # original code from https://levelup.gitconnected.com/writing-tetris-in-python-2a16bddb5318
        # DID not check first row of board (HIGHLY UNLIKELY to ever clear this, but I believe was error)
        for i in range(self.buffer, self.full_height):
            zeros = 0
            for j in range(self.width):
                if self.field[i][j] == 0:
                    zeros += 1
                # breaking DOESN'T SEEM TO WORK-> Not sure why
                # else: 
                #     break
            if zeros == 0:
                lines += 1
                self.lines += 1
                # CHECK RANGE BOUNDARIES, seems to stop at second row 
                # at top
                for i1 in range(i, 1, -1):
                    for j in range(self.width):
                        self.field[i1][j] = self.field[i1 - 1][j]
       
    
        # TO DO: FIX SCORE FUNCTION
        #  Include Gold/Silver blocks
        self.score += lines ** 2

        
    # returns the y value of where piece will drop, used for drawing shadow block
    def shadow_height(self): 
        old_y = self.figure.y    
        while not self.intersects():
            self.figure.y += 1
        self.figure.y -= 1
        new_y = self.figure.y
        self.figure.y = old_y
        return new_y
        
    def go_space(self):
        if self.state == "gameover":
            return
        while not self.intersects():
            self.figure.y += 1
        self.figure.y -= 1
        self.freeze()

    def go_down(self):
        self.figure.y += 1
        if self.intersects():
            self.figure.y -= 1
            self.freeze()

    def freeze(self):
        # for i in range(4):
        #     for j in range(4):
        for ind in self.figure.image():
            i = ind//4
            j = ind%4
                # if i * 4 + j in self.figure.image():
            self.field[i + self.figure.y][j + self.figure.x] = self.figure.color

        self.break_lines()
        self.has_swapped = False
        
        
        # hopefully a better/quicker way to check for gameover
        for j in range(self.width):
            # check row = 3 (4th row), any non zero elements mean Game Over
            if self.field[self.buffer-1][j] != 0: # 
                self.state = "gameover"
                return # no need to plot next figure
                
        self.new_figure()
            

    def swap(self):
        if self.has_swapped:
            # SHOULD play error sound cause you can't swap when you already swapped
            print('You\'ve already swapped, can\'t do it till next piece')
            return
        
        self.has_swapped = True
        # save current piece for later
        if not(self.swap_piece):
            self.swap_piece = Figure(x=self.tet_x,  y=self.tet_y, mode= self.figure.type)
            self.new_figure()
        else:
            temp_piece = Figure(x=self.tet_x,  y=self.tet_y, mode= self.swap_piece.type)
            self.swap_piece = Figure(x=self.tet_x,  y=self.tet_y, mode= self.figure.type)
            self.figure = temp_piece

        

    def go_side(self, dx):
        old_x = self.figure.x
        self.figure.x += dx
        if self.intersects():
            self.figure.x = old_x

    def rotate(self, direction):
        # no rotation for the O block
        if self.figure.name == 'O':
            return
        
        old_rotation = self.figure.rotation
        self.figure.rotate(direction)
        
        # BASE CASE, no kick needed
        if not(self.intersects()):
        #     self.figure.rotation = old_rotation
        # else: 
            return
        
        # Failed without nudge
        #   4 states for each block, not magic number
        new_rotation = (old_rotation + direction)%4 
        
        if self.figure.name == 'I':
            nudges = self.figure.kicks_I[(old_rotation, new_rotation)]
        else:
            nudges = self.figure.kicks_main[(old_rotation, new_rotation)]
        
        old_x = self.figure.x
        old_y = self.figure.y
        
        for nudge in nudges:
            self.figure.x += nudge[0]
            self.figure.y += nudge[1]
            if not (self.intersects()):
                return
            else:
                self.figure.x = old_x
                self.figure.y = old_y
        
        # IF no nudges work
        self.figure.rotation = old_rotation
        

        
# One random function- BETTER SPOT FOR THIS GUY?    
# Transparency for block shadow, code taken from: https://stackoverflow.com/questions/6339057/draw-a-transparent-rectangles-and-polygons-in-pygame
def draw_rect_alpha(surface, color, rect):
    shape_surf = pygame.Surface(pygame.Rect(rect).size, pygame.SRCALPHA)
    pygame.draw.rect(shape_surf, color, shape_surf.get_rect())
    surface.blit(shape_surf, rect)

    
    
    
    
# Initialize the game engine
pygame.init()
# set screen (size set at top 800x800
screen = pygame.display.set_mode(size)
pygame.display.set_caption("Tetris")

# GAME LOOP 
# Loop until the user clicks the close button.
done = False
clock = pygame.time.Clock()
fps = 50
game = Tetris()
counter = 0
pressing_down = False
pressing_sideways = 0



while not done:
    
    # create a new piece if one doesn't exist
    if game.figure is None:
        game.new_figure()
        
    # print(f'counter = {counter}')
    counter += 1
    if counter > 100000:
        counter = 0

    # Every time counter gets to ~ 60//level//2 (or down key)
    # drop piece (TOTALLY WRONG! FIX, somehow works though)
    if counter % (fps // game.level // 2) == 0 or pressing_down:
        if game.state == "start":
            game.go_down()
            
            
            
    # TOO FAST
    if pressing_sideways != 0 and counter%2 == 0: #-1 for left, 1 for right
        game.go_side(pressing_sideways)

    for event in pygame.event.get():
        if event.type == pygame.QUIT:
            done = True
        if event.type == pygame.KEYDOWN:
            if event.key == pygame.K_ESCAPE:
                game.__init__(game.height, game.width)
            if game.state == 'gameover':
                pressing_down = False
                pressing_sideways = 0
                break
            if event.key == pygame.K_RSHIFT:
                game.rotate(direction = 1)
            if event.key == pygame.K_SLASH:
                game.rotate(direction = -1)
            if event.key == pygame.K_DOWN:
                pressing_down = True
            else:
                pressing_down = False  
            if event.key == pygame.K_LEFT:
                # game.go_side(-1)
                pressing_sideways = -1
            elif event.key == pygame.K_RIGHT:
                # game.go_side(1)
                pressing_sideways = 1
            else:
                pressing_sideways = 0
                
                
            if event.key == pygame.K_SPACE:
                game.go_space()
            
            
            if event.key in [pygame.K_0, pygame.K_1, pygame.K_2, 
                             pygame.K_3, pygame.K_4, pygame.K_5, pygame.K_6]:
                game.new_figure(mode = event.key - pygame.K_0) # ASSUME K_0 is 48 and rest of numbers go up by 1
            
            if event.key == pygame.K_s:
                game.swap()
            
    if event.type == pygame.KEYUP:
            if event.key == pygame.K_DOWN:
                pressing_down = False
            if event.key == pygame.K_LEFT or event.key == pygame.K_RIGHT:
                pressing_sideways = 0

    screen.fill(WHITE)

    # Drawing screen
    for i in range(game.buffer, game.full_height):
        for j in range(game.width):
            # WHAT IS ZOOM of 20 doing for the rectangle drawing
            pygame.draw.rect(screen, GRAY, [game.x + game.zoom * j, game.y + game.zoom * i, game.zoom, game.zoom], width = 3)
            if game.field[i][j] > 0 and i >= game.buffer:
                pygame.draw.rect(screen, colors[game.field[i][j]],
                                 [game.x + game.zoom * j + 1, game.y + game.zoom * i + 1, game.zoom - 2, game.zoom - 1])

    # UPDATE#
    if game.figure is not None:
        shadow_y = game.shadow_height()
        for ind in game.figure.image():
            i = ind//4
            j = ind%4
            p = i * 4 + j
            
            # Plotting of shadow piece
            if shadow_y + i >= game.buffer:
                draw_rect_alpha(screen, 
                                tuple(list(colors[game.figure.color])+[TRANSPARENCY]),
                                 [game.x + game.zoom * (j + game.figure.x) + 1,
                                  game.y + game.zoom * (i + shadow_y) + 1,
                                  game.zoom - 2, game.zoom - 2])

            # Plotting of actual piece
            if game.figure.y + i >= game.buffer:
                pygame.draw.rect(screen, colors[game.figure.color],
                                 [game.x + game.zoom * (j + game.figure.x) + 1,
                                  game.y + game.zoom * (i + game.figure.y) + 1,
                                  game.zoom - 2, game.zoom - 2])
                
                
    # Plot SWAP piece if it has been set aside
    if game.swap_piece:
        for ind in game.swap_piece.image():
            i = ind//4
            j = ind%4
            pygame.draw.rect(screen, colors[game.swap_piece.color],
                             [game.swap_x + game.zoom * j,
                              game.swap_y + game.zoom * i,
                              game.zoom - 2, game.zoom - 2]) 
    else:
        # draw something indicating what this spot is for
        pass
    
    fig_i = 0
    for fig in game.queue:
        for ind in fig.image():
            i = ind//4
            j = ind%4
            pygame.draw.rect(screen, colors[fig.color],
                             [game.queue_x + game.zoom * j,
                              game.queue_y + game.zoom * (i + fig_i*5) , # testing coordinates
                              game.zoom - 2, game.zoom - 2]) 

        fig_i += 1
       
    
    
    # Displaying screen text
    font = pygame.font.SysFont('Calibri', 25, True, False)
    font1 = pygame.font.SysFont('Calibri', 65, True, False)
    text = font.render("Score: " + str(game.score), True, BLACK)
    text_lines = font.render("Lines: " + str(game.lines), True, BLACK)
    text_game_over = font1.render("Game Over", True, (255, 125, 0))
    text_game_over1 = font1.render("Press ESC", True, (255, 215, 0))
    text_swap = font.render("SWAP!", True, BLACK)
    text_queue = font.render("Queue:", True, BLACK)
      
    controlsX = (10)
    controlsY = (300)
    position = controlsX, controlsY
    font = pygame.font.SysFont('Calibri', 15)
    text_control = ["Controls",
            "/: CCW rotation",
           "rShift': CW rotation",
           "up,down,left,right: movement",
           "space: hard drop",
           "0-6: debug blocks",
           "s: swap",
           "esc: restart game"]
    label = []
    for line in text_control: 
        label.append(font.render(line, True, GRAY))    
    for line in range(len(label)):
        screen.blit(label[line],(position[0],position[1]+(line*15)+(15*line)))
     
    screen.blit(text, [100, 100])
    screen.blit(text_lines, [100, 150])
    screen.blit(text_swap, [50, 250])
    screen.blit(text_queue, [game.queue_x, game.queue_y-50])
    # screen.blit(text_controls, [50, 300])
    if game.state == "gameover":
        screen.blit(text_game_over, [250, 50])
        screen.blit(text_game_over1, [250, 100])

    pygame.display.flip()
    clock.tick(fps)
pygame.quit()
